In [ ]:
import sklearn 
import gensim 
import os 
cd = os.getcwd(); project_root = f'{cd}'

import sys 
sys.path.append(f'{project_root}/lib')

from text_processor import TextProcessor as TP 
tp = TP(pos=['名詞'])
import pandas as pd 
df = pd.read_csv(f'{project_root}/data/doboku_2019_texts.csv')
print(df)
ndf = df.dropna()
texts = ndf['texts'].map(tp.norm_wakati)

# print(texts)


In [ ]:
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer 
tokenizer = RegexpTokenizer(r'\w\w+')
import many_stop_words
stop_en = many_stop_words.get_stop_words('en')
stop_ja = many_stop_words.get_stop_words('ja')
stop_add = set(open(f'{project_root}/data/stopwords.txt').read().split())
stop_words = stop_en | stop_ja | stop_add
def remove_stop_words(text, stop_words=stop_words):
    return [term for term in text if not term in stop_words]


texts = texts.map(tokenizer.tokenize).map(remove_stop_words)
print(texts)

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=20)


In [ ]:
'''
各トピックの要素
'''
for topic in model.print_topics(num_topics=3, num_words=10):
    print(topic)

In [ ]:
'''
文章がどのトピックに属するか
'''
print(texts.values[0])
print(model.get_document_topics(corpus[0]))



In [ ]:
import numpy as np 
import csv 

tmp = []
for r in model.get_document_topics(corpus):
    tmp2 = []
    j = 0
    for i, v in r:
        while j!=i:
            tmp2.append(0)
            j += 1
        tmp2.append(v)
        j += 1
    while len(tmp2) < 3:
        tmp2.append(0)
    tmp.append(tmp2)


res = np.array(tmp).T 

for i in range(len(res)):
    ndf[f'score_{i}'] = res[i]
print(ndf)
print(df)
df = df.merge(ndf, how='left', on=['texts', 'file_name'])
print(df)
df.to_csv(f'{project_root}/data/doboku_2019_texts_lda.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
        